In [9]:
###load pbmc model
import numpy as np
import pandas as pd
import scanpy as sc
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from iGTP_Linear import *
from iGTP_model_modified_scvi_sample_final import *
from preprocess import *
from learning_utilies import *
import os 
import scanpy as sc
import pickle5 as pickle

In [11]:
pbmc_path='./save_model/pbmc_IFN_1/processed_data/GO_merged_PCzinb_uniform_7000.pkl'

with open(pbmc_path, 'rb') as file:
    pbmc_data = pickle.load(file)
    
print(pbmc_data.keys())

dict_keys(['data_x_s', 'mask_dict', 'args', 'seq_gene_df', 'seq_gene_df_su', 'TP_df', 'ppi_df_1', 'data'])


In [15]:
### find model and load
args = pbmc_data['args']

model_list=os.listdir(args['model_dir'])
model_list=[i for i in model_list if '.ipynb_checkpoints' not in i]
model_list=[i for i in model_list if 'zinb' in i]
model_list=[i for i in model_list if 'best' in i]
model_list.sort()

data_x_s=pbmc_data['data_x_s']
mask_list_dict=pbmc_data['mask_dict']
data=pbmc_data['data']

model=iGTP(args,mask_list_dict)
saved_state_dict = torch.load(args['model_dir']+model_list[1],map_location="cpu")
model.load_state_dict(saved_state_dict)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


<All keys matched successfully>

In [16]:
model.eval()
with torch.no_grad():
    _, p, q=model.encode(torch.tensor(data_x_s).float())

z=q.loc.cpu().detach().numpy()
TP_df=pbmc_data['TP_df']
z_tp=z[:,0:len(TP_df)]

In [17]:
ppi_for_sankey = "BP_response_to_interferon_alpha"

In [18]:
bp_alpha_mad = pd.read_csv("figure_6_result/BP_response_to_interferon_alpha_mad.csv")

In [33]:
bp_alpha_mad.Interaction = bp_alpha_mad.Interaction.str.replace('-', '_')

In [19]:
pbmc_data["TP_df"][pbmc_data["TP_df"]["TP_name"].str.contains(ppi_for_sankey)]

,TP_name,TP_gene
421,BP_response_to_interferon_alpha,"[KLHL20, TPR, IFIT2, IFITM3, PYHIN1, GATA3, IF..."


In [20]:
tp_index = pbmc_data["TP_df"][pbmc_data["TP_df"]["TP_name"].str.contains(ppi_for_sankey)].index[0]

In [21]:
decoder_TP_PPI_mask = model.decoder_TP_PPI[0].state_dict()["mask"]
decoder_TP_PPI_weight = model.decoder_TP_PPI[0].state_dict()["weight"]
decoder_TP_PPI_bias = model.decoder_TP_PPI[0].state_dict()["bias"]

ppi_output = torch.tensor(z_tp[:,tp_index]).unsqueeze(1).mm((decoder_TP_PPI_mask[:,tp_index] * decoder_TP_PPI_weight[:,tp_index]).unsqueeze(0))
ppi_output += decoder_TP_PPI_bias.unsqueeze(0).expand_as(ppi_output)

In [23]:
pbmc_data["ppi_df_1"].head(2)

,gene1,gene2,Interaction,TP_index
615,AASDHPPT,BABAM1,AASDHPPT_BABAM1,0
626,AASDHPPT,JUN,AASDHPPT_JUN,1


In [24]:
decoder_PPI_Gene_mask = model.decoder_PPI_Gene[0].state_dict()["mask"]
decoder_PPI_Gene_weight = model.decoder_PPI_Gene[0].state_dict()["weight"]
decoder_PPI_Gene_bias = model.decoder_PPI_Gene[0].state_dict()["bias"]

In [28]:
ppi_index_BP_response_to_interferon_alpha = []
for i in pbmc_data["TP_df"][pbmc_data["TP_df"]["TP_name"].str.contains(ppi_for_sankey)]["TP_gene"].iloc[0]:
    ppi_index_BP_response_to_interferon_alpha.append(pbmc_data["ppi_df_1"][pbmc_data["ppi_df_1"]["Interaction"].str.contains(i)].TP_index.tolist())

print(ppi_index_BP_response_to_interferon_alpha[0])
###flatten it
ppi_index_BP_response_to_interferon_alpha = [item for sublist in ppi_index_BP_response_to_interferon_alpha for item in sublist]

ppi_index_BP_response_to_interferon_alpha = list(np.unique(ppi_index_BP_response_to_interferon_alpha))

[2233, 8749, 10050, 13127, 16864, 18896, 19236, 20323, 20324, 20325, 20326, 20327, 20328, 20329, 20330, 20331]


In [34]:
pbmc_data["ppi_df_1"][pbmc_data["ppi_df_1"]["Interaction"].isin(bp_alpha_mad.Interaction.tolist())]

,gene1,gene2,Interaction,TP_index
161938,ERBB2,IFITM3,ERBB2_IFITM3,13240
184542,FYN,IFITM3,FYN_IFITM3,15336
188464,GATA3,MTA3,GATA3_MTA3,15691
188488,GATA3,TBX21,GATA3_TBX21,15702
235666,IFITM3,RUVBL2,IFITM3_RUVBL2,18378
235668,IFITM3,SRF,IFITM3_SRF,18379
235669,IFITM3,STAT3,IFITM3_STAT3,18380


In [35]:
ppi_index_BP_response_to_interferon_alpha = pbmc_data["ppi_df_1"][pbmc_data["ppi_df_1"]["Interaction"].isin(bp_alpha_mad.Interaction.tolist())].TP_index.tolist()

In [36]:
gene_output = ppi_output[:,ppi_index_BP_response_to_interferon_alpha].mm((decoder_PPI_Gene_mask[:,ppi_index_BP_response_to_interferon_alpha] * decoder_PPI_Gene_weight[:,ppi_index_BP_response_to_interferon_alpha]).T)
gene_output += decoder_PPI_Gene_bias.unsqueeze(0).expand_as(gene_output)

In [37]:
### select gene group 1
gene_inf_alpha_index_ls = pbmc_data["seq_gene_df"][(pbmc_data["seq_gene_df"].gene.isin(pbmc_data["TP_df"][pbmc_data["TP_df"]["TP_name"].str.contains(ppi_for_sankey)]["TP_gene"].iloc[0])) & (pbmc_data["seq_gene_df"]["gene_group"] == 1)].new_gene_index.tolist()

In [38]:
print(z_tp[:,tp_index].shape)
print(ppi_output[:,ppi_index_BP_response_to_interferon_alpha].shape)
print(gene_output[:,gene_inf_alpha_index_ls].shape)

(28871,)
torch.Size([28871, 7])
torch.Size([28871, 7])


In [39]:
len((decoder_TP_PPI_mask[:,tp_index] * decoder_TP_PPI_weight[:,tp_index]).numpy().tolist())

29266

In [40]:
sum(1 for item in (decoder_TP_PPI_mask[:,tp_index] * decoder_TP_PPI_weight[:,tp_index]).numpy().tolist() if item != 0)

155

In [57]:
###firstly, we try to use parameter as sankey value

mat1 = (decoder_TP_PPI_mask[:,tp_index] * decoder_TP_PPI_weight[:,tp_index])[ppi_index_BP_response_to_interferon_alpha].unsqueeze(0)
mat2 = (decoder_PPI_Gene_mask[:,ppi_index_BP_response_to_interferon_alpha] * decoder_PPI_Gene_weight[:,ppi_index_BP_response_to_interferon_alpha]).T[:,gene_inf_alpha_index_ls]

In [58]:
torch.sum(mat2 != 0, dim=1)

tensor([1, 1, 1, 1, 1, 1, 1])

In [59]:
mat1.shape

torch.Size([1, 7])

In [60]:
num_select_ppi = mat1.shape[1]
mat1 = mat1[:,0:num_select_ppi].abs()
mat2 = mat2[0:num_select_ppi,:].abs()

In [61]:
mat2[torch.nonzero(mat1[0] == 0).squeeze(),:] = 0

In [62]:
# Get the shape of the matrices
rows, cols = mat1.shape

# Initialize empty lists for sources, targets, and values
sources = []
targets = []
values = []

# Node names for layers
layer1_nodes = ["BP_response_to_interferon_alpha"]
layer1_nodes_color=['red']*len(layer1_nodes)
layer2_nodes = [pbmc_data["ppi_df_1"][pbmc_data["ppi_df_1"].TP_index == i]["Interaction"].values[0] for i in ppi_index_BP_response_to_interferon_alpha]
layer2_nodes = layer2_nodes[0:num_select_ppi]
layer2_nodes_color=['blue']*len(layer2_nodes)
layer3_nodes = [pbmc_data["seq_gene_df"][pbmc_data["seq_gene_df"].new_gene_index == i]["gene"].values[0] for i in gene_inf_alpha_index_ls]
layer3_nodes_color=['green']*len(layer3_nodes)

In [63]:
layer3_nodes

['GATA3', 'IFIT2', 'IFIT3', 'IFITM3', 'KLHL20', 'PYHIN1', 'TPR']

In [64]:
len(layer2_nodes)

7

In [65]:
cols

7

In [66]:
# Iterate through the matrices to create connections
for i in range(rows):
    if torch.all(torch.abs(mat1[i]) < 0.05):
        print(i)
        pass
    else:
        for j in range(cols):
            source_node = layer1_nodes[i]  # Node in layer1
            target_node = layer2_nodes[j]  # Node in layer2
            value = mat1[i, j]

            # Only add non-zero values to the Sankey diagram
            #if abs(value) > 0.05:
            if value != 0:
                sources.append(source_node)
                targets.append(target_node)
                values.append(value)

In [67]:
positions_in_layer2nodes = {element: index for index, element in enumerate(layer2_nodes)}
positions_in_layer2nodes = [positions_in_layer2nodes[element] for element in targets]

In [68]:
# Repeat the process for connections from layer2 to layer3
mat2 = mat2[positions_in_layer2nodes,:]
rows, cols = mat2.shape
for i in range(rows):
    if torch.all(torch.abs(mat2[i]) < 0.000001):
        #print(i)
        pass
    else:
        for j in range(cols):
            source_node = layer2_nodes[positions_in_layer2nodes[i]]  # Node in layer2
            target_node = layer3_nodes[j]  # Node in layer3
            value = mat2[i, j]
 
            # Only add non-zero values to the Sankey diagram
            #if abs(value) > 0.05:
            if value != 0:
                sources.append(source_node)
                targets.append(target_node)
                values.append(value)

In [69]:
names = layer1_nodes+layer2_nodes+layer3_nodes
name_to_index = {}
for index, name in enumerate(names):
    name_to_index[name] = index

In [70]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      label = layer1_nodes+layer2_nodes+layer3_nodes,
      color = layer1_nodes_color+layer2_nodes_color+layer3_nodes_color
    ),
    link = dict(
      source = [name_to_index[i] for i in sources],
      target = [name_to_index[i] for i in targets],
      value = values
  ))])

fig.update_layout(title_text=ppi_for_sankey, font_size=14)
fig.show()

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Sample DataFrame
data = {'X': [1, 2, 3, 4, 5],
        'Y': [10, 12, 5, 8, 7],
        'Category': ['A', 'B', 'A', 'B', 'A']}
df = pd.DataFrame(data)

# Define colors for each category
colors = {'A': 'red', 'B': 'blue'}

# Create a Scatter trace for each category
traces = []
for category, color in colors.items():
    trace = go.Scatter(x=df[df['Category'] == category]['X'],
                      y=df[df['Category'] == category]['Y'],
                      mode='markers',
                      marker=dict(color=color),
                      name=category)
    traces.append(trace)

# Create the layout
layout = go.Layout(title='Scatter Plot with Color-Coded Categories',
                   xaxis=dict(title='X'),
                   yaxis=dict(title='Y'))

# Create the figure
fig = go.Figure(data=traces, layout=layout)

# Show the plot
fig.show()
